In [1]:
!pip install git+https://github.com/christophM/rulefit.git

  Cloning https://github.com/christophM/rulefit.git to /tmp/pip-req-build-s5bk67cu
  Running command git clone --filter=blob:none --quiet https://github.com/christophM/rulefit.git /tmp/pip-req-build-s5bk67cu
  Resolved https://github.com/christophM/rulefit.git to commit 472b8574b4eb9e565caf1e05ed580998fe2c9a8e
  Preparing metadata (setup.py) ... done


In [15]:
import numpy as np
import pandas as pd

from rulefit import RuleFit, RuleEnsemble

from sklearn.preprocessing import LabelEncoder



In [3]:

carbon_data = pd.read_csv("res/carbon_emissions.csv", index_col=0)
carbon_data.head()

,Sex,Diet,How Often Shower,Heating Energy Source,Transport,Vehicle Type,Social Activity,Monthly Grocery Bill,Frequency of Traveling by Air,Vehicle Monthly Distance Km,Waste Bag Size,Waste Bag Weekly Count,How Long TV PC Daily Hour,How Many New Clothes Monthly,How Long Internet Daily Hour,Energy efficiency,Recycling,Cooking_With,CarbonEmission
Body Type,,,,,,,,,,,,,,,,,,,
overweight,female,pescatarian,daily,coal,public,NaN,often,230,frequently,210,large,4,7,26,1,No,['Metal'],"['Stove', 'Oven']",2238
obese,female,vegetarian,less frequently,natural gas,walk/bicycle,NaN,often,114,rarely,9,extra large,3,9,38,5,No,['Metal'],"['Stove', 'Microwave']",1892
overweight,male,omnivore,more frequently,wood,private,petrol,never,138,never,2472,small,1,14,47,6,Sometimes,['Metal'],"['Oven', 'Microwave']",2595
overweight,male,omnivore,twice a day,wood,walk/bicycle,NaN,sometimes,157,rarely,74,medium,3,20,5,7,Sometimes,"['Paper', 'Plastic', 'Glass', 'Metal']","['Microwave', 'Grill', 'Airfryer']",1074
obese,female,vegetarian,daily,coal,private,diesel,often,266,very frequently,8457,large,1,3,5,6,Yes,['Paper'],['Oven'],4743


In [4]:
def get_encoded(label, data, encoder):
    le = LabelEncoder()
    le.fit(data[label])
    encoder[label] = le
    return le.transform(data[label])
    

In [5]:
# preprocessing of data:
carbon_data = carbon_data.dropna(how='any') # drop Nan Values for now in a way i dont care

y = carbon_data.CarbonEmission.values

X = carbon_data.drop("CarbonEmission", axis=1)
#X = carbon_data[["Sex", "Diet"]]
features = X.columns
encoder_dict = {}
for label in X.columns:
    if X[label].dtype != "int" or X[label].dtype != "float":
        X[label] = get_encoded(label, X, encoder_dict)


X = X.to_numpy()
X.shape


(3279, 18)

In [6]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators=500, max_depth=10, learning_rate=0.01)
rf = RuleFit(tree_generator=gb, max_iter=1000)

rf.fit(X, y, feature_names=features)

RuleFit(max_iter=1000,
        tree_generator=GradientBoostingRegressor(learning_rate=0.01,
                                                 max_depth=10, max_leaf_nodes=4,
                                                 n_estimators=560,
                                                 random_state=559))

array([2299.30066558, 4106.9143734 , 1913.72622724, ..., 3265.52007908,
       5095.50978651, 1174.40283746])

In [12]:

rf = RuleFit(max_iter=1000)
rf.fit(X, y, feature_names=features)

/home/robin/anaconda3/envs/uu_courses/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.098e+06, tolerance: 4.811e+05
  model = cd_fast.enet_coordinate_descent(


RuleFit(max_iter=1000,
        tree_generator=GradientBoostingRegressor(learning_rate=0.01,
                                                 max_depth=100,
                                                 max_leaf_nodes=9,
                                                 n_estimators=584,
                                                 random_state=583,
                                                 subsample=0.13527762099825857))

In [13]:
rf.predict(X)

array([2396.66604874, 4203.8987328 , 2137.13700125, ..., 3159.94975395,
       5063.45374644, 1135.02952026])

In [19]:

re = RuleEnsemble(rf.tree_generator)

rules = re.rules
#rlf.extract_rules_from_tree(rf.tree_generator)
#rules = rules[rules.coef != 0].sort_values("support", ascending=False)

print(rules)

[9 <= 846.5, 9 > 846.5, 9 <= 1656.5, 9 > 1656.5, 9 <= 1542.0, 9 > 1542.0, 9 <= 2070.5 & 8 <= 2.5 & 5 <= 3.5, 9 <= 2070.5 & 8 <= 2.5 & 5 > 3.5, 9 <= 2070.5 & 8 > 2.5, 9 > 2070.5 & 5 <= 2.5 & 5 <= 0.5, 9 > 2070.5 & 5 <= 2.5 & 5 > 0.5, 9 > 2070.5 & 5 > 2.5, 9 <= 1601.0, 9 > 1601.0, 9 <= 1702.0 & 8 <= 2.5, 9 <= 1702.0 & 8 > 2.5, 9 > 1702.0 & 5 <= 2.5, 9 > 1702.0 & 5 > 2.5, 9 <= 1778.5, 9 > 1778.5 & 5 <= 2.5, 9 > 1778.5 & 5 > 2.5, 9 <= 1756.0 & 8 <= 2.5, 9 <= 1756.0 & 8 > 2.5, 9 > 1756.0 & 5 <= 2.5, 9 > 1756.0 & 5 > 2.5, 9 <= 2129.0, 9 > 2129.0, 9 <= 1572.0, 9 > 1572.0, 9 <= 1706.0, 9 > 1706.0 & 5 <= 2.5 & 5 <= 0.5, 9 > 1706.0 & 5 <= 2.5 & 5 > 0.5, 9 > 1706.0 & 5 > 2.5, 5 <= 2.5, 5 > 2.5, 9 <= 2084.5 & 8 <= 2.5 & 5 <= 3.5, 9 <= 2084.5 & 8 <= 2.5 & 5 > 3.5 & 9 <= 1045.0, 9 <= 2084.5 & 8 <= 2.5 & 5 > 3.5 & 9 > 1045.0, 9 <= 2084.5 & 8 > 2.5, 9 > 2084.5, 9 <= 1979.0, 9 > 1979.0, 9 <= 903.5 & 8 <= 2.5, 9 <= 903.5 & 8 > 2.5, 9 > 903.5 & 5 <= 2.5 & 5 <= 0.5, 9 > 903.5 & 5 <= 2.5 & 5 > 0.5 & 5 <= 1